# Fish-Speech-1.5による日本語テキスト→音声変換アプリ

## 準備
以下のセルを実行して、必要なライブラリをインストールします。初回のみ実行してください。

In [ ]:
# 必要なライブラリのインストール（最初に一度だけ実行）
!pip install torch torchaudio transformers huggingface_hub numpy scipy ipywidgets

## アプリの実装
以下のセルを実行して、テキスト→音声変換アプリを起動します。

In [ ]:
import os
import io
import tempfile
import time
import numpy as np
import torch
from IPython.display import Audio, display, HTML
import ipywidgets as widgets
from huggingface_hub import hf_hub_download
import scipy.io.wavfile as wavfile
from transformers import AutoProcessor, AutoModel
import torchaudio

# モデル名
MODEL_ID = "fishaudio/fish-speech-1.5"

# モデルとプロセッサをロードする関数
def load_model(model_id=MODEL_ID, device="auto"):
    """
    Fish-Speech-1.5モデルとプロセッサをロードする
    
    Parameters:
    -----------
    model_id : str
        Hugging Faceのモデル名
    device : str
        'auto', 'cpu', 'cuda', 'cuda:0'などの使用デバイス
        
    Returns:
    --------
    tuple
        (プロセッサ, モデル)
    """
    print(f"モデル {model_id} をロード中...")
    
    # デバイスを自動選択
    if device == "auto":
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    print(f"使用デバイス: {device}")
    
    # プロセッサとモデルのロード
    processor = AutoProcessor.from_pretrained(model_id)
    model = AutoModel.from_pretrained(model_id).to(device)
    
    print("モデルのロードが完了しました")
    return processor, model

# テキストを音声に変換する関数
def text_to_speech(text, processor, model, speaker_id=None, speed=1.0, device="auto"):
    """
    日本語テキストを音声に変換する
    
    Parameters:
    -----------
    text : str
        変換する日本語テキスト
    processor : AutoProcessor
        テキスト処理用プロセッサ
    model : AutoModel
        Fish-Speech-1.5モデル
    speaker_id : int
        話者ID（Noneの場合はランダム選択）
    speed : float
        発話速度（1.0が標準）
        
    Returns:
    --------
    str
        生成された音声ファイルのパス
    """
    # デバイスを自動選択
    if device == "auto":
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 一時ファイルを作成
    temp_dir = tempfile.gettempdir()
    output_path = os.path.join(temp_dir, 'output.wav')
    
    # 言語を検出（簡易的な実装 - 実際はモデルは自動検出可能）
    # 主に日本語の文字を含むかチェック
    has_japanese = any([ord(char) > 0x3000 for char in text])
    language = "ja" if has_japanese else "en"
    
    # テキストからモデル入力を生成
    inputs = processor(
        text=text,
        voice_preset=speaker_id,  # Noneの場合はランダム
        language=language,
        return_tensors="pt"
    ).to(device)
    
    # 音声生成
    with torch.no_grad():
        output = model.generate(**inputs, do_sample=True)
    
    # 出力を取得
    audio_array = output.cpu().numpy().squeeze()
    
    # 発話速度の調整（リサンプリング）
    if speed != 1.0:
        # サンプリングレートはモデルに依存（一般的には24000Hz）
        sampling_rate = 24000
        
        # リサンプリングの比率を計算
        resample_ratio = 1.0 / speed
        new_sampling_rate = int(sampling_rate * resample_ratio)
        
        # リサンプリング
        audio_tensor = torch.tensor(audio_array).unsqueeze(0)
        resampler = torchaudio.transforms.Resample(
            orig_freq=sampling_rate,
            new_freq=new_sampling_rate
        )
        resampled_audio = resampler(audio_tensor).squeeze().numpy()
        
        # 元のサンプリングレートでファイルに保存
        wavfile.write(output_path, sampling_rate, resampled_audio.astype(np.float32))
    else:
        # 変更なしでファイルに保存
        wavfile.write(output_path, 24000, audio_array.astype(np.float32))
    
    return output_path

# ファイルからテキストを読み込む関数
def read_text_file(uploaded_file):
    """
    アップロードされたファイルからテキストを読み込む
    
    Parameters:
    -----------
    uploaded_file : UploadedFile
        アップロードされたテキストファイル
        
    Returns:
    --------
    str
        ファイルの内容
    """
    content = uploaded_file.read()
    # エンコーディングを自動検出（日本語ファイルの場合、utf-8やshift-jisなど様々な可能性がある）
    encodings = ['utf-8', 'shift-jis', 'euc-jp', 'iso-2022-jp']
    
    for encoding in encodings:
        try:
            text = content.decode(encoding)
            return text
        except UnicodeDecodeError:
            continue
    
    # どのエンコーディングでも失敗した場合
    raise ValueError("ファイルのエンコーディングを検出できませんでした。")

# モデルとプロセッサをロード
processor, model = load_model()

# 話者IDのオプション
speaker_options = [
    ("ランダム", None),
    ("日本語女性1", "ja_female_1"),
    ("日本語女性2", "ja_female_2"),
    ("日本語男性1", "ja_male_1"),
    ("日本語男性2", "ja_male_2")
]

# ファイルアップロード時の処理
def on_upload_change(change):
    """
    ファイルがアップロードされたときの処理
    """
    if not change.new:
        return
    
    # 処理状況を表示
    status_output.value = "ファイルを処理中..."
    
    try:
        # アップロードされたファイルからテキストを読み込む
        uploaded_file = list(change.new.values())[0]
        text = read_text_file(uploaded_file)
        
        # テキストプレビューを表示
        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
        
        # テキストを音声に変換
        speed = float(speed_slider.value)
        speaker_id = speaker_dropdown.value
        audio_path = text_to_speech(text, processor, model, speaker_id=speaker_id, speed=speed)
        
        # 音声を表示
        audio_output.clear_output()
        with audio_output:
            display(Audio(audio_path))
        
        # ダウンロードリンクを作成
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        
        download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')
        
        status_output.value = "処理完了！"
    
    except Exception as e:
        status_output.value = f"エラーが発生しました: {str(e)}"

# ダウンロードリンクを作成する関数
def create_download_link(audio_data, filename, text):
    """
    音声データをダウンロードするためのHTMLリンクを作成
    
    Parameters:
    -----------
    audio_data : bytes
        ダウンロードするオーディオデータ
    filename : str
        ダウンロード時のファイル名
    text : str
        リンクに表示するテキスト
        
    Returns:
    --------
    str
        HTMLリンク
    """
    b64 = io.BytesIO(audio_data)
    payload = b64.getvalue()
    import base64
    b64_str = base64.b64encode(payload).decode()
    return f'<a href="data:audio/wav;base64,{b64_str}" download="{filename}">{text}</a>'

# 速度変更時の処理
def on_speed_change(change):
    """
    速度スライダーが変更されたときの処理
    """
    speed_value.value = f"発話速度: {change.new}x"

# UIの作成
upload_button = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='テキストファイルを選択'
)

speaker_dropdown = widgets.Dropdown(
    options=speaker_options,
    value=None,
    description='話者:',
)

speed_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='速度:',
    continuous_update=False
)

speed_value = widgets.HTML(value="発話速度: 1.0x")
status_output = widgets.HTML(value="ファイルをアップロードしてください。")
text_preview = widgets.Textarea(
    description='テキスト:',
    placeholder='アップロードされたテキストがここに表示されます',
    disabled=True,
    layout=widgets.Layout(width='100%', height='200px')
)

audio_output = widgets.Output()
download_link = widgets.HTML()

# イベントハンドラの登録
upload_button.observe(on_upload_change, names='value')
speed_slider.observe(on_speed_change, names='value')

# UIの表示
display(widgets.HTML("<h1>Fish-Speech-1.5 日本語テキスト→音声変換アプリ</h1>"))
display(widgets.HTML("<p>日本語のテキストファイルをアップロードして、自然な音声に変換します。</p>"))
display(widgets.HBox([upload_button, speaker_dropdown, widgets.VBox([speed_slider, speed_value])]))
display(status_output)
display(text_preview)
display(widgets.HTML("<h3>生成された音声:</h3>"))
display(audio_output)
display(download_link)

# 直接テキスト入力機能も追加
text_input = widgets.Textarea(
    description='直接入力:',
    placeholder='ここに直接テキストを入力して変換することもできます',
    layout=widgets.Layout(width='100%', height='150px')
)

# テキスト入力による変換ボタン
convert_button = widgets.Button(
    description='テキストを変換',
    button_style='primary',
    tooltip='入力されたテキストを音声に変換します'
)

# ボタンクリック時の処理
def on_convert_button_click(b):
    text = text_input.value
    if not text:
        status_output.value = "テキストが入力されていません。"
        return
    
    status_output.value = "テキストを処理中..."
    
    try:
        # テキストプレビューを表示
        text_preview.value = text if len(text) <= 1000 else text[:1000] + "..."
        
        # テキストを音声に変換
        speed = float(speed_slider.value)
        speaker_id = speaker_dropdown.value
        audio_path = text_to_speech(text, processor, model, speaker_id=speaker_id, speed=speed)
        
        # 音声を表示
        audio_output.clear_output()
        with audio_output:
            display(Audio(audio_path))
        
        # ダウンロードリンクを作成
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
        
        download_link.value = create_download_link(audio_data, 'output.wav', '音声ファイルをダウンロード')
        
        status_output.value = "処理完了！"
    
    except Exception as e:
        status_output.value = f"エラーが発生しました: {str(e)}"

# イベントハンドラの登録
convert_button.on_click(on_convert_button_click)

# 直接入力UIの表示
display(widgets.HTML("<h3>または直接テキストを入力:</h3>"))
display(text_input)
display(convert_button)